In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import pymongo
import urllib.parse
import dns
from mongoengine import *
from mongoengine.context_managers import switch_collection
import random
import sys

In [3]:
PATH = r"C:\Users\BILAL\Projects\LinkedInScraper\chromedriver.exe"
driver = webdriver.Chrome(PATH)

In [4]:
sleeps = [2,3,4]
driver.get("https://www.linkedin.com/login")
driver.find_element_by_id("username").send_keys('donnybegins@gmail.com')
password = driver.find_element_by_id("password")
time.sleep(random.choice(sleeps))
password.send_keys('myaccount')
password.send_keys(Keys.RETURN)

In [44]:
def chatWindow():
        try:
            chatOpen = driver.find_element_by_xpath("//*[@id='msg-overlay']/div[1]/div")
            personal_chatOpen = driver.find_element_by_xpath("//div[@id='msg-overlay__emoji-hoverable-outlet']")

            if chatOpen:
                chatClose = driver.find_element_by_xpath("(//*[@id='msg-overlay']/div[1]/header/section[1])")
                chatClose.click()
                print("Chatbox Closed...")
                return True
            if personal_chatOpen:
                close_personal_chat = driver.find_element_by_xpath("//button[@data-control-name='overlay.close_conversation_window']")
                close_personal_chat.click()
                print("Chat closed")
                return True
                
        except:
            print("Chatbox is already closed")
            return True

In [6]:
feed = driver.current_url

In [7]:
chatWindow()
search = driver.find_element_by_class_name("search-global-typeahead__input ")
search.send_keys('Java')
search.send_keys(Keys.RETURN)
print("Entered keyword")
time.sleep(10)

Chatbox Closed...
Entered keyword


In [8]:
people = driver.find_element_by_xpath("//button[@aria-label='People']")
people.click()
time.sleep(random.choice(sleeps))
#selecting location(India)
time.sleep(10)
container = driver.find_element_by_xpath("//button[@aria-label='Locations filter. Clicking this button displays all Locations filter options.']")
container.click()
time.sleep(random.choice(sleeps))

enter_location = driver.find_element_by_xpath("//input[@placeholder='Add a location']")
# enter_location
enter_location.send_keys('India')
time.sleep(5)

find_location = driver.find_element_by_xpath("//div[@role='listbox']")
print(find_location)
select_location = find_location.find_element_by_xpath("//div[@role='option']")
select_location.click()

select_location = driver.find_elements_by_xpath("//div[@class='artdeco-hoverable-content__shell']//button[@aria-label='Apply selected filter and show results']")
select_location[1].click()

<selenium.webdriver.remote.webelement.WebElement (session="77e31addc1512a2991f22d800feed499", element="26a15057-2474-4909-b2fe-de5f288b4bd8")>


In [9]:
url = driver.current_url
pagination = url
pagination

'https://www.linkedin.com/search/results/people/?geoUrn=%5B%22102713980%22%5D&keywords=Java&origin=FACETED_SEARCH'

In [ ]:
pagination = f"{pagination}&page={page}"
print("Pagination", pagination)
driver.get(pagination)
chatWindow()

In [45]:
chatWindow()

Chatbox is already closed


True

In [46]:
close_pchat = driver.find_element_by_xpath("//button[@data-control-name='overlay.close_conversation_window']")
close_pchat

<selenium.webdriver.remote.webelement.WebElement (session="77e31addc1512a2991f22d800feed499", element="012c6554-0ac7-4df2-b958-62f5550007a2")>

In [47]:
close_pchat.click()

In [13]:
chatClose = driver.find_element_by_xpath("(//*[@id='msg-overlay']/div[1]/header/section[1])")
chatClose.click()

In [22]:
length = driver.execute_script("return document.documentElement.scrollHeight")
for i in range(100, length, 150):
        driver.execute_script("window.scrollTo(0, " + str(i) + ")")
        time.sleep(2)

In [24]:
driver.execute_script("window.scrollTo("+str(length)+",0)")

In [26]:
result_list = driver.find_elements_by_xpath("//li[@class='reusable-search__result-container ']")
len(result_list)

10

In [28]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [32]:
container = soup.find("div", attrs={"role": "main"})
profile = container.find("li")
profile_link = profile.find('a')['href']

'https://www.linkedin.com/in/kalyani-zanwar'

In [39]:
profiles = container.find_all("li")

In [37]:
profiles = container.find_all("li")
profiles[0].find('a')['href']

'https://www.linkedin.com/in/kalyani-zanwar'

In [40]:
for enu, result in enumerate(result_list):
#     print(profiles[enu].find('a')['href'])
    print(result)
    connect = result.find_element_by_xpath("//button[@class='artdeco-button artdeco-button--2 artdeco-button--secondary ember-view']")
    connect.click()
    print("Clicking on Connect")
    time.sleep(random.choice(sleeps))
    print("Sending request...")

    #adding Note
    add_note = driver.find_element_by_xpath("//button[@aria-label='Add a note']")
    add_note.click()
    time.sleep(random.choice(sleeps))
    add_note = driver.find_element_by_xpath("//textarea[@name='message']")
    add_note.send_keys(" ")
    time.sleep(random.choice(sleeps))

    #sending note
    send_req = driver.find_element_by_xpath("//button[@aria-label='Send now']")
    send_req.click()
    print("Request Sent")
    profile_link = profiles[enu].find('a')['href']
    driver.get(profile_link)
    break

<selenium.webdriver.remote.webelement.WebElement (session="77e31addc1512a2991f22d800feed499", element="b77d4b1a-9c15-440b-bed5-37f6449efbff")>
Clicking on Connect
Sending request...
Request Sent


In [41]:
link = f"{driver.current_url}detail/contact-info"
link

'https://www.linkedin.com/in/kalyani-zanwar/detail/contact-info'

In [54]:
def get_contact_info():
    
    sites = []
    phone = ''
    email = ''
    ims = []
    detail = []
    extra_links = []
    #getting to contact-info page
    driver.get(link)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, features="html.parser")
    time.sleep(random.choice(sleeps))
    
    #getting name
    names = soup.find("div", {"class": "artdeco-modal__header ember-view"})
    name = names.find("h1").text.strip()
    
    #contact sections
    container = soup.find("div", attrs={"class": "pv-profile-section__section-info section-info"})
    sections = container.find_all("section")
    
    for section in sections:
                text = section.find('header').text.strip()
                print(text)
                if "Profile" in text:
                    linkedin_url = section.find('a')['href']
                    print(linkedin_url)
                    continue
                if "Website" in text:
                    websites = section.find_all('li')

                    for website in websites:
                        print(website.find('a')['href'])
                        sites.append(website.find('a')['href'])
                    continue
                if text == "Phone":
                    phone = section.find("span").text
                    print(phone)
                    continue
                if text == 'Email':
                    email = section.find('a')['href']
                    email = email.split(':')[1:]
                    email = ''.join(email)
                    email = email.strip()
                    print(email)
                    continue
                if text == "IM":
                    im = section.find_all("li")
                    for i in im:
                        spans = i.find_all("span")

                        for span in spans:
                            detail.append(span.text.strip())
                        ims = '-'.join(detail)
#                         print(ims)
                    continue
                if "Connected" in text:
                    continue
                extra_link = section.find('a')
                if extra_link is not None:
                    extra_link = extra_link['href']
#                     print(extra_link)
                    extra_links.append(extra_link)

    if len(sites) == 0:
        sites = ''
    if len(ims) == 0:
        ims = ''
    if extra_links == 0:
        extra_links = ''
    driver.get(link)
    
    return name, linkedin_url, email, phone, sites, ims, extra_links

In [52]:
name, linkedin_url, email, phone, sites, ims, extra_links = get_contact_info()

Kalyani’s Profile
https://www.linkedin.com/in/kalyani-zanwar


In [182]:
def get_experience_info():
    length = driver.execute_script("return document.documentElement.scrollHeight")
#     for i in range(100, length, 200):
#         driver.execute_script("window.scrollTo(0, " + str(i) + ")")
#         time.sleep(2)
    soup = BeautifulSoup(driver.page_source, features="html.parser")
    time.sleep(random.choice(sleeps))
    exp = soup.find('section', attrs={"id": "experience-section"})
#     print(exp)
    if exp != None:
        exp = exp.find('ul').find_all('li')
#         print(exp)
        experience = []
        for ex in exp:
#             print('exppp')

            
#             print("first designation",designation)
#             if designation == -1:
#                 continue
#             if 'Company Name' in designation:
#                 desig = ex.find('ul')
#                 designation = desig.find('h3').text.strip()
#                 designation = designation.replace('\n', ' ')
#                 designation = designation.split(' ')[1:]
#                 designation = ' '.join(designation)
# #                 print(designation)
                
#                 company = ex.find('h3').text.strip()
#                 company = company.replace('\n', ' ')
#                 company = company.split(' ')[2:]
#                 company = ' '.join(company)
# #                 print(company)
                
#             if designation == None:
#                 designation = ''




            company = ex.find('p', {"class": "pv-entity__secondary-title t-14 t-black t-normal"})
            if company == -1:
                continue
            company = ex.find('p', {"class": "pv-entity__secondary-title t-14 t-black t-normal"})
#             print(company)
            if company == None:
                company = ''
            else:
                company = company.text.strip()
            if 'time' in company:
                company = company.split()[:-1]
                company = ' '.join(company)
            if '\n' in company:
                company = company.replace('\n', '')
                company = company.split()[0]
                
#             print(company)
            duration = ex.find('h4')
#             print(duration)
            if duration == None:
                duration = ''
                print("NONE")
            else:
                duration = duration.text.strip()
                print(duration)
                duration = duration.replace('\n', ' ')
                print(duration)
                duration = duration.split(' ')[2:]
                print(duration)
                duration = ' '.join(duration)
                print(duration)
                print("##############")
            
            
            designation = ex.find('h3')
            
            if designation == None:
                designation = ''
            else:
                designation = designation.text.strip()
                designation = designation.replace('\n', ' ')
            
#             print("first designation",designation)
            if designation == -1:
                continue
            if 'Company Name' in designation:
                desig = ex.find('ul')
                designation = desig.find('h3').text.strip()
                designation = designation.replace('\n', ' ')
                designation = designation.split(' ')[1:]
                designation = ' '.join(designation)
#                 print(designation)
                
                company = ex.find('h3').text.strip()
                company = company.replace('\n', ' ')
                company = company.split(' ')[2:]
                company = ' '.join(company)
#                 print(company)
#                 if duration == None:
#                     duration = ''
#                     print("NONE")
#                 else:
#                     duration = duration.text.strip()
#                     print(duration)
#                     duration = duration.replace('\n', ' ')
#                     print(duration)
#                     duration = duration.split(' ')[2:]
#                     print(duration)
#                     duration = ' '.join(duration)
#                     print(duration)


            experience.append({"Company": company, "Designation": designation, "Duration": duration})

    else:
        experience = "No details found"

    return experience

In [183]:
experience = get_experience_info()


Total Duration
2 yrs 3 mos
Total Duration 2 yrs 3 mos
['2', 'yrs', '3', 'mos']
2 yrs 3 mos
##############
Full-time
Full-time
[]

##############
Internship
Internship
[]

##############


In [179]:
print(experience)

[{'Company': '18North Systems Private Limited', 'Designation': 'Software Developer', 'Duration': '2 yrs 3 mos'}, {'Company': '', 'Designation': 'Title Software Developer', 'Duration': ''}, {'Company': '', 'Designation': 'Title Software Developer', 'Duration': ''}]


In [150]:
get_experience_info()

Software Developer
18North Systems Private Limited

222 2 yrs 3 mos

222 

222 


In [66]:
experience = get_experience_info()

In [67]:
experience

[{'Company': '',
  'Designation': 'Company Name\n18North Systems Private Limited',
  'Duration': 'yrs3mos'},
 {'Company': '', 'Designation': 'Title\nSoftware Developer', 'Duration': ''},
 {'Company': '', 'Designation': 'Title\nSoftware Developer', 'Duration': ''}]